## Histopathologic CNN Analyzer

This architecture is based on the LeNet network as described in Dive Into Deep Learning (2021, Aston Zhang, Zachary C. Lipton, Mu Li, and Alexander J. Smola), Section 6.6.1.

It has been substantially modified to incorporate improvements from the literature.

In [ ]:
import torch.nn as nn
import torchvision.transforms as tx

In [ ]:
class Histopath_CNN(nn.Module):

    def __init__(self, debug):
        super(Histopath_CNN, self).__init__()
        
        self.debug = debug

        # Image Pre-Processing
        self.conv0 = nn.Sequential(
            nn_debug(debug),
            # Add standard center cropping based on numerous trials of larger and 
            #   smaller data values for the Histopathologic Cancer dataset.
            #   The labelled portion of the images are within the 32x32 center.
            #TODO probably move out to pre-processing pipeline, but this requires
            #   replacement of the input data, not addition to.
            tx.CenterCrop(54)
        )
        self.conv1 = nn.Sequential(
            # Print input shape
            nn_debug(debug),
            # Convolutional layer
            nn.Conv2d(3, 12, kernel_size=5, padding=2),
            # Normalize values across the batch; wants a larger batch size
            #   e.g. > 50; may aid training by reducing internal covariate shift -
            #   making it less sensitive to spatial arrangement of parameters
            nn.BatchNorm2d(12),
            # Activation function - non-linear function - ReLU offers better 
            #  training performance due to its less gentle curve than other actives
            nn.ReLU(),
            # Pooling 'steps back' from the details a little bit in order to help
            #   generalize the image
            nn.MaxPool2d(kernel_size=2, stride=2)
      )
        self.conv2 = nn.Sequential(
            nn_debug(debug),
            nn.Conv2d(12, 24, kernel_size=5),
            nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            # Collapse the 3D data into a 1D input for the linear units
            nn.Flatten()
        )
        # Traditional linear network for classification learning
        self.fc1 = nn.Sequential(
            nn_debug(debug),
            nn.Linear(2904, 4240),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn_debug(debug),
            nn.Linear(4240, 1228),
            nn.ReLU()
       )
        self.fc3 = nn.Sequential(
            nn_debug(debug),
            nn.Linear(1228, 1),
            # Apply Sigmoid activation in order to get a smoother probability
            #   curve, which is required by the BinaryCrossEntropy Loss function
            nn.Sigmoid()
        )

    def forward(self, x):
        # Image tx
        x=self.conv0(x)

        # convolutions
        x = self.conv1(x)
        x = self.conv2(x)

        # FC layers
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x

In [ ]:
class nn_debug(nn.Module):
    def __init__(self, debug):
        super(nn_debug, self).__init__()
        self.debug = debug

    def forward(self, x):
        if self.debug:
            print(f'## Input: {x.shape}')
        return x